In [ ]:
# 异步爬虫实战
# 现有网站内容已进行加密

'''
1. 同步操作: 访问getCatalog 拿到所有章节的cid和名称
2. 异步操作: 访问getChapterContent 下载所有文章内容
'''
import json
import asyncio
import aiohttp
import aiofiles
import requests


async def aiodownload(cid,b_id,title):
    data = {
        "book_id":b_id,
        "cid":f"{b_id}|{cid}",
        "need_bookinfo":1
    }
    data = json.dumps(data) # 把json变化为字符串
    url = f"http:564156?data={data}"

    async with aiohttp.ClientSession() as session:
        async with session.get(url) as resp:
            dic = await resp.json()

            async with aiofiles.open(title, mode='w',encoding='utf-8') as f:
                await f.write(dic['data']['novel']['content'])


def getCatalog(url):
    resp = requests.get(url)
    dic = resp.json()
    tasks = []
    for item in dic['data']['novel']['items']: # item就是的对应的每一个章节的名称和cid
        title = item['title']
        cid = item['cid']
        # 准备异步任务
        d = asyncio.create_task(aiodownload(cid,b_id,title))
        tasks.append(d)

    await asyncio.wait(tasks)


if __name__ == '__main__':
    b_id = "4306063500"
    url = '章节的内容{id}'.format(id=b_id)
    asyncio.run(getCatalog(url))